In [5]:
import heapq

def abnormal_finder(g_enzyme):
    '''This function finds the extended base in the G-enzyme split without G in it. 
    It represents the last fragment in the final RNA sequences.'''
    check = 'G'
    this = [i for i in g_enzyme if check not in i]
    return this

def G_CU_splitter(g_enzyme):
    '''Splits the G-enzyme on the CU enzyme'''
    new_list = []

    for string in g_enzyme:
        modified_string = ''
        for char in string:
            modified_string += char
            if (char in ('U', 'C')) and (string.index(char) != len(string)-1):
                modified_string += '-'

        new_list.append(modified_string)

    return new_list


def CU_G_splitter(CU_enzyme):
    '''Splits the UC-enzyme on the G-enzyme'''
    new_list = []

    for string in CU_enzyme:
        modified_string = ''
        for char in string:
            modified_string += char
            if (char in ('G')) and (string.index(char) != len(string)-1):
                modified_string += '-'

        new_list.append(modified_string)

    return new_list


def finder(first, second):
    '''Finds all extended bases that are by themselves not attached to any other extended bases.'''
    singles = []
    
    final=first+second
    for i in final:
        if i.find('-') == -1:
            singles.append(i)
    
    return singles


def path_finder(first, second, abnormal, start_end):
    '''Returns all non-single fragments in g-uc enzyme split and the uc-g enzyme split. 
        And appends the beginning fragment onto the end fragment. Also returns the end fragment by itself as well.'''
    paths = []
    end = ''
    for i in start_end:
        if i in abnormal[0]:  #could be potential problem.. so like U in CU... but AU not in CU... but what if abnormal is AU.. them U and AU are both in it.. so what do I consider as last?... here the program would fail... so this is a condition that needs to be satisfied.. put this in final ouput screen.
            end = end + i
            start_end.remove(i) 
    final=first+second
    for i in final:
        if i.find('-') != -1: 
            paths.append(i)
        if paths[-1].split('-')[-1] ==end:
            paths[-1]= paths[-1]+'*'+start_end[0]
            
    
    
    return paths, end


def interior_vertice_finder(first, second):
    '''This returns all the interior extended bases of the extended bases. '''
    interior_vertices = []
    final = first+second
    for i in final:
        temp = i.split('-')
        temp.pop(len(temp)-1)
        if len(temp)>0:
            temp.pop(0)  
        
        interior_vertices+=temp
    
    return interior_vertices

def start_end_func(edges, vertices):
    '''Considering their are always to more single extended bases than interior extended bases. 
    This is going to return the start and end vertices of the final sequences.'''
    this = edges
    for i in vertices:
        this.remove(i)
    
    return this

def creation(vertices, end, this, paths):
    #final_vertices = [i for i in vertices if i != end] #I should loop through paths to collect all begginnings and ends.. and then have these as final vertices... using this method could be prone to errors if my procedure is not accruate
    #size=len(final_vertices)
    #matrix  = [[[] for _ in range(size) ] for _ in range(size)] 
    #for i,j in enumerate(final_vertices): #this dictionary should be formedd by using paths and not final_vertices... it is safer and adheres to structure better incase my implication is wrong!
    #    this[j]=i
    
    #['AU-C-G', 'AU-G', 'C-U*AU', 'AC-U-AU-AC-G', 'G-G-AC', 'G-AU', 'G-C']
    #['C', 'U', 'AU', 'AC', 'G']
    final_vertices = set()
    final = set()
    for i in paths:
        act_vertices = i.split('-')
        final_vertices.add(act_vertices[0])
        final_vertices.add(act_vertices[-1])
    for i in final_vertices:
        if i.find('*')!= -1:
            curr = i.split('*')
            real_vert = curr[-1]
            final.add(real_vert)
        else:
            final.add(i)
    size=len(final)
    matrix  = [[[] for _ in range(size) ] for _ in range(size)] 
    for i,j in enumerate(final): 
        this[j]=i
        
        
        
    
    
    total_edge_counter = 0
    x=0
    for i in paths:
        if ('*' not in i):
            curr = i.split('-')
            out = this[curr[0]] 
            inward = this[curr[-1]]
            curr.remove(curr[0])
            curr.remove(curr[-1])
            if len(curr)==0:
                matrix[out][inward].append(str(x))
                total_edge_counter+=1
                x+=1
            elif len(curr) != 0:
                matrix[out][inward].append('-'.join(curr))
                total_edge_counter+=1
    
    
        elif('*' in i):
            curr = i.split('*')
            incoming=curr[-1]
            
            ind=curr[0].find('-') 
            adding=curr[0][ind+1:] 
            
            outgoing = curr[0].replace(end,'').replace('-','') 
            curr.remove(curr[-1])
            matrix[this[outgoing]][this[incoming]].append(adding)
            total_edge_counter+=1
            
    
    return final, matrix, total_edge_counter, this



def eulerian_finder(end, start_end, matrix, total_edge_counter, dicter):
    
    start_end=start_end[0]
    reverse = {value: key for key, value in dicter.items()}
    paths = []
    queue = [([dicter[start_end]], [], [])]
    
    while queue:
        node, path, edges = heapq.heappop(queue)
        if ((node[-1] == end) and (len(edges)==total_edge_counter)):
            paths.append(path)
            continue
        elif(node[-1]==end and (len(edges) != total_edge_counter)):
            continue
            
        path.append(reverse[node[-1]])
        
        for i, j in enumerate(matrix[node[-1]]):
            if len(j) != 0:
                for x in j:
                    if x not in edges and x != end:
                        new_node = node + [i]
                        new_path = path + [x]  
                        new_edges = edges + [x]
                        heapq.heappush(queue, (new_node, new_path, new_edges))
                    elif x not in edges and x == end:
                        new_node = node + [x]
                        new_path = path + [x]  
                        new_edges = edges + [x]
                        heapq.heappush(queue, (new_node, new_path, new_edges))
    return paths


def cleaner(paths):
    proper = []
    for i in paths:
        curr_path = ''
        for j in i:
            j=j.replace('-','')
            if j.isupper() and j.isalpha():
                curr_path += j
        proper.append(curr_path)
    return proper


def main():
    
    
    user_input_G = input("Enter g-enzyme split (all caps and space before next subseqquence): ")
    split_input = user_input_G.split(', ')
    g_enzyme_split = [string.strip() for string in split_input]



    user_input_CU = input("Enter CU-enzyme split (all caps and space before next subseqquence): ")
    split_input = user_input_CU.split(', ')
    CU_enzyme_split = [string.strip() for string in split_input]
    
    print(f" the g enzyme ssplit : {g_enzyme_split}")
    abnormal_extendedBase = abnormal_finder(g_enzyme_split)
    
    print(f" this is the abnormal extended base: {abnormal_extendedBase}")

    G_then_UC_split  = G_CU_splitter(g_enzyme_split)
    print(f"G then UC split: {G_then_UC_split}")
    
    UC_then_G_split = CU_G_splitter(CU_enzyme_split)
    print(f"UC then G enzyme split : {UC_then_G_split}")
    
    edge_cases = finder(G_then_UC_split, UC_then_G_split)
    print(f"the singles-potential edge cases for graph : {edge_cases}")
    
    interior_vertices = interior_vertice_finder(G_then_UC_split, UC_then_G_split)
    print(f" this is the interior-singles-vertices : {interior_vertices}")  
    
    start_end = start_end_func(edge_cases, interior_vertices)
    print(f" this is start and end vertices: {start_end}")                    
    
    paths, end = path_finder(G_then_UC_split, UC_then_G_split, abnormal_extendedBase, start_end) 
    print(f" these are potential routes : {paths}, and end is : {end}")
    
    final_vertices, matrix, total_edge_counter, dicter = creation(interior_vertices, end, {}, paths)
    print(f"this is the final: {final_vertices} \n {matrix} \n {total_edge_counter} \n {dicter}")
    
    eulerian_paths = eulerian_finder(end, start_end, matrix, total_edge_counter, dicter)
    print(f"the paths : {eulerian_paths}")
    
    
    proper_paths  = cleaner(eulerian_paths)
    print(f"cleaned up paths: ", proper_paths)
    return

if __name__ == "__main__":
    main()

'''AUCG, AUG, G, CU, ACUAUACG is the g-enzyme split and GGAC, U, AU, GAU, C, U, AC, GC, AU is the UC enzyme split. 
Now their is only one potential problem. This has to do with two extra fragments in the singles extended bases when 
compared to the interiors extended bases. These two fragments are always the start and end. But disambiguating between 
the start and end is not clear. For this reason, for the above code to work, the following assumptions must be made.
1: The two extra fragments (in the singles versus the interior extended bases) have to be in the interior extended bases 
and have to be in the non single fragments. 2: The abnormal fragment from the g-enzyme has to carry in it either of the two extra 
fragments to be able to disambiguate between the start and end fragments. This disambiguation must make it clear as to which fragment
is the start and which is the end. If either of these assumptions fails then the code crumbles.'''






 the g enzyme ssplit : ['AUCG', 'AUG', 'G', 'CU', 'ACUAUACG']
 this is the abnormal extended base: ['CU']
G then UC split: ['AU-C-G', 'AU-G', 'G', 'C-U', 'AC-U-AU-AC-G']
UC then G enzyme split : ['G-G-AC', 'U', 'AU', 'G-AU', 'C', 'U', 'AC', 'G-C', 'AU']
the singles-potential edge cases for graph : ['G', 'U', 'AU', 'C', 'U', 'AC', 'AU']
 this is the interior-singles-vertices : ['C', 'U', 'AU', 'AC', 'G']
 this is start and end vertices: ['U', 'AU']
 these are potential routes : ['AU-C-G', 'AU-G', 'C-U*AU', 'AC-U-AU-AC-G', 'G-G-AC', 'G-AU', 'G-C'], and end is : U
this is the final: {'AU', 'G', 'AC', 'C'} 
 [[[], ['C', '0'], [], []], [['1'], [], ['G'], ['2']], [[], ['U-AU-AC'], [], []], [['U'], [], [], []]] 
 7 
 {'AU': 0, 'G': 1, 'AC': 2, 'C': 3}
the paths : [['AU', '0', 'G', '1', 'AU', 'C', 'G', 'G', 'AC', 'U-AU-AC', 'G', '2', 'C', 'U'], ['AU', 'C', 'G', '1', 'AU', '0', 'G', 'G', 'AC', 'U-AU-AC', 'G', '2', 'C', 'U'], ['AU', '0', 'G', 'G', 'AC', 'U-AU-AC', 'G', '1', 'AU', 'C', 'G', '2', 

'2 big problems: 1) the way the matrix is constructed based off the vertices is faulty I think. \nIt should be built by using the paths and by taking the non single paths.\nThen the matrix and the vertices and the inner fragments in the matrix are better off.\n2) the second problem is with this idea.. The abnormal fragment is always their, and the two extra end and beggingin fragments I guess \nare always there.... but which fragment is end and beggining needs to be better disambiugated. If this is solved \nin a stronger manner then the paths and then matrix and then construction of RNA sequences is a lot stronger!!'